In [ ]:
import quadrotor

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
# we define a few constants
xdim = 6 # dimension of the state (p,v)
udim = 2 # dimension of the control (u)
N = 300 # number of steps
nvars = N*udim + (N+1)*xdim # total number of variables
dt = 0.01 #delta t


# Define the Quadrotor Dynamics
A = quadrotor.A
B = quadrotor.B


## We define the cost
Q = np.eye(xdim)
Q[0,0] = Q[2,2] = Q[4,4] = 1000.  # position penalty
Q[1,1] = Q[3,3] = Q[5,5] = 100.   # velocity penalty
R = np.eye(udim) * 0.01

# Desired position
x_desired = np.array([3, 0, 3, 0, 0, 0])

## we create G
def get_cost_matrix_G(Q, R):
    '''
    we are making this a function so we can reuse this later in the next cells when we play with costs
    '''
    G = np.zeros((nvars, nvars))
    for i in range(N):
        G[i*(xdim+udim):i*(xdim+udim)+xdim,i*(xdim+udim):i*(xdim+udim)+xdim] = Q
        G[i*(xdim+udim)+xdim:i*(xdim+udim)+xdim+udim, i*(xdim+udim)+xdim:i*(xdim+udim)+xdim+udim] = R
    G[N*(xdim+udim):N*(xdim+udim)+xdim,N*(xdim+udim):N*(xdim+udim)+xdim] = Q
    return G

G = get_cost_matrix_G(Q,R)

##### CREATE THE KKT SYSTEM #####
## create the linear equality constraint
C = np.zeros([xdim * (N+1), nvars])
d = np.zeros([xdim * (N+1)])

C[0:2, 0:2] = np.eye(xdim)
d[0] = 10. # initial position of the car

A = np.array([[1, dt],[0,1]])
B = np.array([[0], [dt]])

for i in range(N):
    ##u
    C[(i+1)*xdim:(i+2)*xdim, i*(xdim+udim)+xdim:i*(xdim+udim)+xdim+1] = B
    ##x n+1
    C[(i+1)*xdim:(i+2)*xdim, (i+1)*(xdim+udim):(i+1)*(xdim+udim)+xdim] = -np.eye(xdim)
    ## xn
    C[(i+1)*xdim:(i+2)*xdim, i*(xdim+udim):(i)*(xdim+udim)+xdim] = A

## create the KKT matrix
nconstraints = C.shape[0]
KKT = np.zeros((nvars+nconstraints, nvars+nconstraints))

KKT[0:nvars, 0:nvars] = G
KKT[nvars:, 0:nvars] = C
KKT[0:nvars, nvars:] = C.T

f = np.zeros((nvars+nconstraints,1))
f[nvars:,0] = d

##### SOLVE THE KKT SYSTEM #####
x = np.linalg.solve(KKT, f)

### read the answers back to p,v,u
car_position = x[0:nvars:3]
car_velocity = x[1:nvars:3]
control = x[2:nvars:3]

plt.figure(figsize=[12,9])
plt.subplot(3,1,1)
plt.ylabel('position')
plt.plot(car_position, 'x')
plt.subplot(3,1,2)
plt.ylabel('velocity')
plt.plot(car_velocity, 'x')
plt.subplot(3,1,3)
plt.plot(control, 'x')
plt.ylabel('acceleration')